-------

## 12-16

### 10-12

1. 面试
2. 会议
3. 一键配置: 自研开发, 耗时好力, 扩展功能灵活
4. 一键配置: OpenStack是一个开源的云计算管理平台, 标准统一, 大规模
5. 一键配置: 滴滴云快照, 快捷省事, 会产生一点费用 (推荐使用)
6. 远程监控: plesk专业主机管理软件, 面向Web服务, 不免费
7. 远程监控: StatsD, serverstatus 都是notejs方面的
8. 远程监控: Nagios 监控系统, 商业软件, 免费监控7个终端
9. 远程监控: Shenkin 重新实现Nagios免费, 相对复杂, 插件较多, 功能繁琐

验证两台主机信任是否已成功建立

Nagios的主要功能特点：
- 监视网络服务 (SMTP, POP3, HTTP, NNTP, PING等)
- 监视主机资源 (进程, 磁盘等)
- 简单的插件设计可以轻松扩展Nagios的监视功能
- 服务等监视的并发处理
- 错误通知功能 (通过email, pager, 或其他用户自定义方法)
- 可指定自定义的事件处理控制器
- 可选的基于浏览器的WEB界面以方便系统管理人员查看网络状态，各种系统问题，以及日志等等
- 可以通过手机查看系统监控信息

### 10-13

1. 一键配置滴滴云快照方式在P4机器验证可行
2. consul 官方文档及软件功能分析
  - 服务发现
  - 健康检测
  - 键值存储
  - 安全服务通信
  - 多数据中心
  
  (文档中没有发现有关控制的功能, 欠缺灵活性, 扩展不便, 所以及时上报异常状态和控制服务的功能需要单独设计) 
    
3. 登陆认证
- 数据传输安全(TLS)
- 还需考虑整机下线

(远程监控方案: 采用consul的检测 + 实现上报/控制)

https://www.consul.io/docs/intro

Consul使用gossip协议管理成员关系、广播消息到整个集群


![GOSSIP](gossip.gif "gossip")

### 10-14

1. UiPath (Stadio, Fabric(process), Doc U)
2. 百度EasyDL (只提供访问接口, 没有模型设计)
3. 阿里PAI Stadio (数据预处理, 特征工程, ML)
4. DeepCognitonAI (无探测)
5. microsoft azure (ML有拖拽, DL以代码形式)
6. machinelearningforkids (没找到模型如何建立的, 但是训练后的模型可以在Scratch应用)
   - 从另一种方式教孩子学习AI
7. Machine Learning Playground (传统ML, 有意思, 自己创数据, 选择模型, 及时看到结果)

https://www.uipath.com/product/platform/rpa-ai-integration-with-ai-fabric

https://www.uipath.com/blog/easier-artificial-intelligence-starter-machine-learning-models

> While AI has become a shorthand for a range of big data, statistical, and predictive techniques, UiPath focuses on “practical AI,” applying AI in tangible scenarios that improve automation outcomes. We’ve built AI into every part of the UiPath Platform, from detecting objects using computer vision, to discovering automation opportunities, and managing and scaling AI. In turn, businesses are moving from simpler automation scenarios to more complex ones, which involve unstructured data, lots of variables, and uncertain outcomes.

> You don’t need to be a data scientist to use ML models
UiPath makes it easy for RPA developers and business users to benefit from AI and ML models without deep data science expertise through products such as UiPath AI Fabric and UiPath Document Understanding.

> With AI Fabric and Document Understanding, ML model retraining has been democratized for non-data scientists, enabling RPA developers and business users with domain-specific knowledge to easily mark up samples. Then they can use AI Fabric to retrain the model without needing to understand or think about feature engineering, hyperparameterization, or any other data scientist-specific knowledge.

Blue Prism
这是一款RPA软件，可为其提供灵活的数字化劳动力组织.它可以从 https://www.blueprism.com/下载.

Automation Anywhere
RPA的另一个重要工具是Automation Anywhere，它可以从 https://www.automationanywhere.com/

UiPath
这是一个RPA软件.它可以帮助组织有效地自动化流程.它可以从 https://www.uipath.com/

1. [machinelearningforkids](https://machinelearningforkids.co.uk/)

  数据标注一体话
  
2. [Machine Learning Playground](https://ml-playground.com)

  所见即所得

https://www.stevemurch.com/machine-learning-ai-for-kids-resources/2018/12

### 10-15 至 10-18

1. 成都出差

    - 参观活动会场
    - '爱阅读'书城查看儿童书籍, 优学派儿童产品了解
    - 签署战略协议

---------------------

## 19-23

### 10-19

1. consul集群设计思路

> GPU机器按区域划分, 每个区域下的所有GPU节点中选一个节点作为sever代理, 该区域其他节点作为client代理, 每个代理节点(GPU机器)都会启动若干个框架相关的服务, client节点将服务状态汇总所在区域的server代理, 所有区域server代理节点在选举出一个leader, 其他作为follower(保证不会应该单一server节点挂了,监控功能被废), 可能的问题: server节点理论上不能太多, 如果区域多了, 这个方案可能需要调整

![](consul-client-server.png)

https://github.com/sunweisheng/spring-cloud-example

```

                                          (wlan/lan)                    (wlan/lan)
                         +--------------+             +--------------+              +--------------+
                         |              | ----------> |              | -----------> |              |
                         |  Server:chi  |             |  Server:eta  |              | Server:sigma |
                         |              | <---------- |              | <----------- |              |
                         +--------------+             +--------------+              +--------------+
                           (follower)                    (leader)                      (follower)
                               ^
                               |
                               | (lan: gossip)
                               |
                               |
    +--------------------------|------------------------+  选择3个机器作为consul server模式
    |                                                   |
    |  +-------------------+     +-------------------+  |  按区域分配机器, 每个gpu机器节点启动consul client模式
    |  |  Client:001-001   |     |  Client:001-002   |  |
    |  |        |          |     |        |          |  |  每个gpu节点启动多个微服务(cv/nlp等)向client agent注册
    |  |  ai/cv/ml/rl/nlp  |     |  ai/cv/ml/rl/nlp  |  |
    |  |    (services)     |     |    (services)     |  |  client agent把消息同步到consul server端
    |  +-------------------+     +-------------------+  |
    |                                                   |  服务监控就可以统一做到server端
    |                +-------------------+              |
    |                |  Client:001-003   |              |
    |                |        |          |              |
    |                |  ai/cv/ml/rl/nlp  |              |
    |                |    (services)     |              |
    |                +-------------------+              |
    |                                                   |
    +---------------------------------------------------+
```

### 10-20

1. 模拟测试consul集群, 基本测试通, 但在测试选举leader时, server节点多会出现混乱

2. 真实环境部署consul server (http://116.85.5.40:8500/)

3. consul client真实环境尚没有测试(明天需要滴滴云开些账号)

4. consul monitor check 方案可行性研究(通过)

https://www.techdrabble.com/hashicorp/37-use-a-consul-watch-to-monitor-vault-seal-status

### 10-21

1. 手机短信发送测试 (筑望平台消息内容必须含有"【】验证码 "字符, 否则发不出去)
  - python脚本测试ok
  - (失败)尝试使用shell实现短息发送(docker consul没有python运行环境)
  - (方案1) 不使用docker中consul, 直接在使用原程序, 重新修改原逻辑. 缺点: 没有always restart能力
  - (方案2:采用) 做一个新consul docker含有python环境
  
2. client代理环境配置及问题:
  - (已解决)快照方式分配的机器, 解决nvidia驱动内置不配问题(WARNING! Diff between built and installed module!)
  - 框架服务crash状态触发的脚本执行存在问题


### 10-22

1. (已解决)failed inserting node: Error while renaming Node ID (node-id暂时使用固定TODO)

2. (已完成)构建带python环境的consul, 短信发送采用python3自带的request, 减少安装依赖

3. (已解决)ai服务注册方式由代码硬编码改为配置方式, 避免consul挂掉无法唤起ai服务注册

4. (未解决)检测异常服务, 监控脚本会多次被执行问题

    - 方案: 在监控脚本中增加判断是否为leader(1/3)的节点, 由leader发送消息避免重复 (未完成)

5. (未解决)所有节点运行一段时间, 概率出现所有节点都退出的问题

### 10-23

1. (已解决)检测异常服务, 监控脚本会多次被执行问题

2. (未解决)服务reload或者异常重启时, 所有watch监控的服务脚本会被触发(BUG)

    - 临时特殊处理一下, 待官网更新

    - issue [#7616](https://github.com/hashicorp/consul/pull/7616)
    
3. (已解决)所有节点启动过程会报服务check状态, 虽然服务是启动的, 但仍处在不可达状态, 导致误报

    - TODO: 触发异常处理脚本, 脚本中二次check服务状态, 防止上报错误状态信息.
    
4. (已完成)短信上报功能

    - TODO: 短信内容大小有限制,多个节点同时上报服务异常, 短信发不出去

-----------------

## 26-30

### 10-26

- (已解决)consul定时退出重启问题 (功能由定时check服务冲突导致)

### 10-27

- 增加自定义网络为空的错误提示 (推荐直接由前端直接判断)

- 增加consul记录用户训练错误信息 (暂没做清除机制, 会产生数据积累, 产品稳定后删除此功能)

- 在服务启动时, 同时启动调试环境下的www服务, 开发测试需要

- 增加用户训练异常信息发送邮件通知功能

- (未完成) ssd代码分析 (yatengLG/SSD-Pytorch 和 lufficc/SSD)

- https://towardsdatascience.com/review-ssd-single-shot-detector-object-detection-851a94607d11

- https://github.com/hashicorp/consul/issues/8687 (解决consul data modify trigger Consul 1.9)

### 10-28

- yatengLG/SSD-Pytorch (代码耦合严重,不适合模块化)

- 增加获取用户训练任务实际占用的GPU, 如果大于设置的阀值页面提示

- Redis消息添加过期时间(5天)

- 联调多输入层生成proto层顺序乱问题

### 10-29

- 分析ssds.pytorch 和 a-PyTorch-Tutorial-to-Object-Detection(代码单一,容易拆分)

- 定位自定义模型输入的层顺序问题(尚未解决,前端逻辑判断复杂)

- 分析生成自定义网络部分代码, 看是否能够节省内存

  - 在forward时是每个层对应一个独立变量, 成倍增大了内存使用(目前还没有解决方案)

- 任务2 连线不完整的情况下依旧可以进行训练
   - (新需求) tabpage跳转模型检查保存问题   

- 任务2 线性层/全连接层不能根据输出层自动调节，需要手动修改参数
   - (非问题)

- 任务2 在训练进行中选择暂停训练 再选择开始训练 系统显示bug
   - (前端) 暂时训练后"开始训练"按钮置灰

- 连续点击继续训练/暂停训练按钮会报bug, 提示框队列展示问题
   - (体验) 优先级低, 暂不解决

- 任务3 模型搭建完成之后训练进程不动了
   - (BUG) 已解决

- 删线逻辑有问题(多输入层, 如果删除连线再次连接, 生成proto层序列问题)
   - (前端) 优先级高, 难度有点大 **尚未解决**

### 10-30

